[단어 통합]
- 바이그램, 트라이그램 방법으로 각각 진행 예정

[불용어 제거]
- 불용어를 추가한 다음, 추가한 불용어를 제거할 예정

# 단어 통합
## 바이그램

In [1]:
import pandas as pd
import numpy as np
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
import re

import konlpy
from konlpy.tag import Okt

# 경고제어
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('everytime_review_data.csv')

In [3]:
df.head()

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,리뷰길이,리뷰_형태소
0,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,28,"['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', ..."
1,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...,22년 2학기 수강자,4,0,131,"['그저', '그래요', '.', '다른건', '다', '낫', '배드', '했는데..."
2,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강... 퀴즈 2번 + 기...,22년 2학기 수강자,4,0,232,"['기', '창', '강의', '평', '피드백', '없고', '강의', '안도',..."
3,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,22년 2학기 수강자,3,0,96,"['과제', '하나', '도', '없고', '출석', '퀴즈', '2', '번', ..."
4,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,KCU 5과목 정도 수강했는데 그중에 제일 별로.. 퀴즈2번 기말고사1번 수업에서 ...,22년 2학기 수강자,3,0,167,"['KCU', '5', '과목', '정도', '수강', '했는데', '그', '중'..."


In [4]:
reviews = df[['리뷰']]
reviews.head()

,리뷰
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움
1,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강... 퀴즈 2번 + 기...
3,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...
4,KCU 5과목 정도 수강했는데 그중에 제일 별로.. 퀴즈2번 기말고사1번 수업에서 ...


- 정규표현식으로 한글 영어 제외하고 제거

In [5]:
# 정규표현식으로 한글, 영어, 공백 제외하고 특수문자 제거
reviews['리뷰'] = [re.sub(r"[^ㄱ-ㅣ가-힣a-zA-Z\s]", '', s) for s in reviews['리뷰']]

In [6]:
reviews['바이그램'] = 0
for i in range(len(reviews['리뷰'])):
    bigs = []
    bigram_plus = []
    a = reviews['리뷰'][i]
    tokens = word_tokenize(a)
    bigram = bigrams(tokens)
    for j in bigram:
        bigs.append(j)

    for k in range(len(bigs)):
        big = bigs[k][0] + " " + bigs[k][1]
        bigram_plus.append(big)
    s = ", ".join(bigram_plus)
    reviews['바이그램'][i] = s

In [7]:
reviews

,리뷰,바이그램
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫 kcu, kcu 강의, 강의 퀄리티, 퀄리티 떨어지지만, 떨어지지만 에이쁠, 에..."
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 문제가 나왔어요 비중도 제일 ...,"그저그래요 다른건, 다른건 다, 다 낫배드했는데, 낫배드했는데 기말고사로는, 기말고..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 번 기말 시험 ...,"기창 강의평, 강의평 피드백, 피드백 없고, 없고 강의안도, 강의안도 없지만, 없지..."
3,과제 하나도 없고 출석 퀴즈번 기말번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록도 ...,"과제 하나도, 하나도 없고, 없고 출석, 출석 퀴즈번, 퀴즈번 기말번만, 기말번만 ..."
4,KCU 과목 정도 수강했는데 그중에 제일 별로 퀴즈번 기말고사번 수업에서 안 배운 ...,"KCU 과목, 과목 정도, 정도 수강했는데, 수강했는데 그중에, 그중에 제일, 제일..."
...,...,...
121583,최고의 교수님 두 학기 연속으로 들었는데 학점 정말 잘 주신다 항상 학생들을 먼저 ...,"최고의 교수님, 교수님 두, 두 학기, 학기 연속으로, 연속으로 들었는데, 들었는데..."
121584,학점 절대평가인 것 같고 비대면이라 좋았음 에릭 교수님 이번 학기가 마지막이라 강의...,"학점 절대평가인, 절대평가인 것, 것 같고, 같고 비대면이라, 비대면이라 좋았음, ..."
121585,에릭교수님 점수 후하게 잘 주시고 좋습니다,"에릭교수님 점수, 점수 후하게, 후하게 잘, 잘 주시고, 주시고 좋습니다"
121586,영회는 에릭 교수님이 짱이다 진심으로 영어를 못해도 잘 이끌어주시고 점점 영어회화에...,"영회는 에릭, 에릭 교수님이, 교수님이 짱이다, 짱이다 진심으로, 진심으로 영어를,..."


### 트라이그램

In [8]:
a = reviews['리뷰'][0]
tokens = word_tokenize(a)
trigram = ngrams(tokens,3)

In [9]:
for t in trigram:
    print(t)

('첫', 'kcu', '강의')
('kcu', '강의', '퀄리티')
('강의', '퀄리티', '떨어지지만')
('퀄리티', '떨어지지만', '에이쁠')
('떨어지지만', '에이쁠', '받기')
('에이쁠', '받기', '쉬움')


In [10]:
reviews['트라이그램'] = 0
for i in range(len(reviews['리뷰'])):
    tries = []
    trigram_plus = []
    a = reviews['리뷰'][i]
    tokens = word_tokenize(a)
    trigram = ngrams(tokens,3)
    for j in trigram:
        tries.append(j)

    for k in range(len(tries)):
        tri = tries[k][0] + " " + tries[k][1] + " " + tries[k][2]
        trigram_plus.append(tri)
    s = ", ".join(trigram_plus)
    reviews['트라이그램'][i] = s

In [11]:
reviews

,리뷰,바이그램,트라이그램
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫 kcu, kcu 강의, 강의 퀄리티, 퀄리티 떨어지지만, 떨어지지만 에이쁠, 에...","첫 kcu 강의, kcu 강의 퀄리티, 강의 퀄리티 떨어지지만, 퀄리티 떨어지지만 ..."
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 문제가 나왔어요 비중도 제일 ...,"그저그래요 다른건, 다른건 다, 다 낫배드했는데, 낫배드했는데 기말고사로는, 기말고...","그저그래요 다른건 다, 다른건 다 낫배드했는데, 다 낫배드했는데 기말고사로는, 낫배..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 번 기말 시험 ...,"기창 강의평, 강의평 피드백, 피드백 없고, 없고 강의안도, 강의안도 없지만, 없지...","기창 강의평 피드백, 강의평 피드백 없고, 피드백 없고 강의안도, 없고 강의안도 없..."
3,과제 하나도 없고 출석 퀴즈번 기말번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록도 ...,"과제 하나도, 하나도 없고, 없고 출석, 출석 퀴즈번, 퀴즈번 기말번만, 기말번만 ...","과제 하나도 없고, 하나도 없고 출석, 없고 출석 퀴즈번, 출석 퀴즈번 기말번만, ..."
4,KCU 과목 정도 수강했는데 그중에 제일 별로 퀴즈번 기말고사번 수업에서 안 배운 ...,"KCU 과목, 과목 정도, 정도 수강했는데, 수강했는데 그중에, 그중에 제일, 제일...","KCU 과목 정도, 과목 정도 수강했는데, 정도 수강했는데 그중에, 수강했는데 그중..."
...,...,...,...
121583,최고의 교수님 두 학기 연속으로 들었는데 학점 정말 잘 주신다 항상 학생들을 먼저 ...,"최고의 교수님, 교수님 두, 두 학기, 학기 연속으로, 연속으로 들었는데, 들었는데...","최고의 교수님 두, 교수님 두 학기, 두 학기 연속으로, 학기 연속으로 들었는데, ..."
121584,학점 절대평가인 것 같고 비대면이라 좋았음 에릭 교수님 이번 학기가 마지막이라 강의...,"학점 절대평가인, 절대평가인 것, 것 같고, 같고 비대면이라, 비대면이라 좋았음, ...","학점 절대평가인 것, 절대평가인 것 같고, 것 같고 비대면이라, 같고 비대면이라 좋..."
121585,에릭교수님 점수 후하게 잘 주시고 좋습니다,"에릭교수님 점수, 점수 후하게, 후하게 잘, 잘 주시고, 주시고 좋습니다","에릭교수님 점수 후하게, 점수 후하게 잘, 후하게 잘 주시고, 잘 주시고 좋습니다"
121586,영회는 에릭 교수님이 짱이다 진심으로 영어를 못해도 잘 이끌어주시고 점점 영어회화에...,"영회는 에릭, 에릭 교수님이, 교수님이 짱이다, 짱이다 진심으로, 진심으로 영어를,...","영회는 에릭 교수님이, 에릭 교수님이 짱이다, 교수님이 짱이다 진심으로, 짱이다 진..."


In [12]:
df['바이그램'] = reviews['바이그램']
df['트라이그램'] = reviews['트라이그램']
df.head()

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,리뷰길이,리뷰_형태소,바이그램,트라이그램
0,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,28,"['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', ...","첫 kcu, kcu 강의, 강의 퀄리티, 퀄리티 떨어지지만, 떨어지지만 에이쁠, 에...","첫 kcu 강의, kcu 강의 퀄리티, 강의 퀄리티 떨어지지만, 퀄리티 떨어지지만 ..."
1,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,그저그래요. 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요. 비중도 ...,22년 2학기 수강자,4,0,131,"['그저', '그래요', '.', '다른건', '다', '낫', '배드', '했는데...","그저그래요 다른건, 다른건 다, 다 낫배드했는데, 낫배드했는데 기말고사로는, 기말고...","그저그래요 다른건 다, 다른건 다 낫배드했는데, 다 낫배드했는데 기말고사로는, 낫배..."
2,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강... 퀴즈 2번 + 기...,22년 2학기 수강자,4,0,232,"['기', '창', '강의', '평', '피드백', '없고', '강의', '안도',...","기창 강의평, 강의평 피드백, 피드백 없고, 없고 강의안도, 강의안도 없지만, 없지...","기창 강의평 피드백, 강의평 피드백 없고, 피드백 없고 강의안도, 없고 강의안도 없..."
3,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,과제 하나도 없고 출석 퀴즈2번 기말1번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록...,22년 2학기 수강자,3,0,96,"['과제', '하나', '도', '없고', '출석', '퀴즈', '2', '번', ...","과제 하나도, 하나도 없고, 없고 출석, 출석 퀴즈번, 퀴즈번 기말번만, 기말번만 ...","과제 하나도 없고, 하나도 없고 출석, 없고 출석 퀴즈번, 출석 퀴즈번 기말번만, ..."
4,기업가정신과창업(KCU),3,이론,x,전학년,교양,교양,김한수,KCU 5과목 정도 수강했는데 그중에 제일 별로.. 퀴즈2번 기말고사1번 수업에서 ...,22년 2학기 수강자,3,0,167,"['KCU', '5', '과목', '정도', '수강', '했는데', '그', '중'...","KCU 과목, 과목 정도, 정도 수강했는데, 수강했는데 그중에, 그중에 제일, 제일...","KCU 과목 정도, 과목 정도 수강했는데, 정도 수강했는데 그중에, 수강했는데 그중..."


In [13]:
df.to_csv('바이그램트라이그램합친review.csv',encoding='utf-8-sig', index=False)

In [14]:
reviews

,리뷰,바이그램,트라이그램
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫 kcu, kcu 강의, 강의 퀄리티, 퀄리티 떨어지지만, 떨어지지만 에이쁠, 에...","첫 kcu 강의, kcu 강의 퀄리티, 강의 퀄리티 떨어지지만, 퀄리티 떨어지지만 ..."
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 문제가 나왔어요 비중도 제일 ...,"그저그래요 다른건, 다른건 다, 다 낫배드했는데, 낫배드했는데 기말고사로는, 기말고...","그저그래요 다른건 다, 다른건 다 낫배드했는데, 다 낫배드했는데 기말고사로는, 낫배..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 번 기말 시험 ...,"기창 강의평, 강의평 피드백, 피드백 없고, 없고 강의안도, 강의안도 없지만, 없지...","기창 강의평 피드백, 강의평 피드백 없고, 피드백 없고 강의안도, 없고 강의안도 없..."
3,과제 하나도 없고 출석 퀴즈번 기말번만 하면됨 퀴즈는 인터넷 서치해야하고 강의록도 ...,"과제 하나도, 하나도 없고, 없고 출석, 출석 퀴즈번, 퀴즈번 기말번만, 기말번만 ...","과제 하나도 없고, 하나도 없고 출석, 없고 출석 퀴즈번, 출석 퀴즈번 기말번만, ..."
4,KCU 과목 정도 수강했는데 그중에 제일 별로 퀴즈번 기말고사번 수업에서 안 배운 ...,"KCU 과목, 과목 정도, 정도 수강했는데, 수강했는데 그중에, 그중에 제일, 제일...","KCU 과목 정도, 과목 정도 수강했는데, 정도 수강했는데 그중에, 수강했는데 그중..."
...,...,...,...
121583,최고의 교수님 두 학기 연속으로 들었는데 학점 정말 잘 주신다 항상 학생들을 먼저 ...,"최고의 교수님, 교수님 두, 두 학기, 학기 연속으로, 연속으로 들었는데, 들었는데...","최고의 교수님 두, 교수님 두 학기, 두 학기 연속으로, 학기 연속으로 들었는데, ..."
121584,학점 절대평가인 것 같고 비대면이라 좋았음 에릭 교수님 이번 학기가 마지막이라 강의...,"학점 절대평가인, 절대평가인 것, 것 같고, 같고 비대면이라, 비대면이라 좋았음, ...","학점 절대평가인 것, 절대평가인 것 같고, 것 같고 비대면이라, 같고 비대면이라 좋..."
121585,에릭교수님 점수 후하게 잘 주시고 좋습니다,"에릭교수님 점수, 점수 후하게, 후하게 잘, 잘 주시고, 주시고 좋습니다","에릭교수님 점수 후하게, 점수 후하게 잘, 후하게 잘 주시고, 잘 주시고 좋습니다"
121586,영회는 에릭 교수님이 짱이다 진심으로 영어를 못해도 잘 이끌어주시고 점점 영어회화에...,"영회는 에릭, 에릭 교수님이, 교수님이 짱이다, 짱이다 진심으로, 진심으로 영어를,...","영회는 에릭 교수님이, 에릭 교수님이 짱이다, 교수님이 짱이다 진심으로, 짱이다 진..."


In [26]:
for i in range(0, 1000):
    print(reviews['바이그램'][i])

첫 kcu, kcu 강의, 강의 퀄리티, 퀄리티 떨어지지만, 떨어지지만 에이쁠, 에이쁠 받기, 받기 쉬움
그저그래요 다른건, 다른건 다, 다 낫배드했는데, 낫배드했는데 기말고사로는, 기말고사로는 서술형, 서술형 문제가, 문제가 나왔어요, 나왔어요 비중도, 비중도 제일, 제일 큰데, 큰데 점수, 점수 어디서, 어디서 깎였는지도, 깎였는지도 모르고, 모르고 다른거에서, 다른거에서 다, 다 만점받고, 만점받고 기말에서, 기말에서 점인가, 점인가 깎였는데, 깎였는데 아슬아슬하게, 아슬아슬하게 에이쁠받았어요, 에이쁠받았어요 에이쁠, 에이쁠 못, 못 받았으면, 받았으면 억울했을듯
기창 강의평, 강의평 피드백, 피드백 없고, 없고 강의안도, 강의안도 없지만, 없지만 그래도, 그래도 할만한, 할만한 꿀강, 꿀강 퀴즈, 퀴즈 번, 번 기말, 기말 시험, 시험 번, 번 퀴즈, 퀴즈 내용, 내용 풀, 풀 때, 때 강의에서, 강의에서 못, 못 본, 본 내용이다, 내용이다 싶으면, 싶으면 검색해서, 검색해서 꼭, 꼭 찾아보길, 찾아보길 그리고, 그리고 시험은, 시험은 서술형으로, 서술형으로 쓸, 쓸 수도, 수도 있으니까, 있으니까 강의, 강의 듣다가, 듣다가 중요하다고, 중요하다고 생각하는, 생각하는 부분, 부분 체크하고, 체크하고 긴, 긴 서술형으로, 서술형으로 쓰면, 쓰면 어떻게, 어떻게 풀어서, 풀어서 쓸지를, 쓸지를 생각해보길, 생각해보길 추천함, 추천함 사이트, 사이트 LMS, LMS 아니니까, 아니니까 꼭, 꼭 웹, 웹 즐겨찾기, 즐겨찾기 추가해두고, 추가해두고 기간, 기간 내, 내 출석하게, 출석하게 생각날, 생각날 때마다, 때마다 들어가세여
과제 하나도, 하나도 없고, 없고 출석, 출석 퀴즈번, 퀴즈번 기말번만, 기말번만 하면됨, 하면됨 퀴즈는, 퀴즈는 인터넷, 인터넷 서치해야하고, 서치해야하고 강의록도, 강의록도 없고, 없고 다른, 다른 과목에, 과목에 비해, 비해 점수, 점수 확인도, 확인도 어렵고, 어렵고 피드백, 피드백 전혀, 전혀 안해주심, 안해주심 하지만, 